In [1]:
import os

import numpy as np
import pandas as pd
from sklearn import preprocessing
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit
from sklearn import metrics
from sklearn import linear_model
from sklearn.preprocessing import LabelEncoder
import gc
import time

/Users/jeong/anaconda3/envs/rpy/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
train_transaction = pd.read_csv('../input/train_transaction.csv')
test_transaction = pd.read_csv('../input/test_transaction.csv')

train_identity = pd.read_csv('../input/train_identity.csv')
test_identity = pd.read_csv('../input/test_identity.csv')
sub = pd.read_csv('../input/sample_submission.csv')

train = train_transaction.merge(train_identity, how='left', on = 'TransactionID')
test = test_transaction.merge(test_identity, how='left', on = 'TransactionID')

In [3]:
del train_identity, test_identity 

In [15]:
print(train.shape)
print(test.shape)

(590540, 434)
(506691, 433)


In [21]:
train['id_05'].unique()

array([ nan,   0.,   1.,   3.,   2.,   9.,  12.,   6.,   4.,  21.,   7.,
         5.,  18.,  17.,  -1.,  11.,   8.,  10., -10.,  -8.,  40.,  13.,
        19.,  -3.,  14.,  22.,  -4.,  15.,  23.,  24.,  -2.,  41.,  20.,
        -5.,  31.,  50.,  -9.,  25.,  27.,  -7.,  33., -26., -11.,  26.,
        32.,  42.,  29.,  -6.,  16., -21., -12., -16.,  28.,  34., -30.,
        30.,  39., -28.,  38., -27.,  37., -13., -15., -14., -18.,  52.,
        35., -17., -23., -25.,  48., -20., -31.,  36., -19.,  47., -34.,
       -24.,  43.,  46.,  45.,  44., -33., -22., -29., -72., -37., -35.,
       -44., -40., -32., -46., -38., -39.])

In [25]:
# test 데이터에는 없는 값들 뽑기
# unique값이 100 미만일때(categorical이라고 가정)
a = list(set(train['id_05'].unique()) - set(test['id_05'].unique()))
a

[nan, 47.0, -46.0, 50.0, -40.0, -72.0, -39.0, -37.0]

In [28]:
a = [x for x in a if str(x) != 'nan']

In [29]:
a

[47.0, -46.0, 50.0, -40.0, -72.0, -39.0, -37.0]

In [31]:
tr2 = train.loc[~train['id_05'].isin(a)]

In [32]:
tr2.shape

(590524, 434)

In [43]:
tr2 = train

In [44]:
for i in range(1, 10) :
    col = 'id_0'+str(i)
    if train[col].nunique() < 100 :
        a = list(set(train[col].unique()) - set(test[col].unique()))
        a = [x for x in a if str(x) != 'nan']
        tr2 = tr2.loc[~tr2[col].isin(a)]

In [45]:
print(train.shape)
print(tr2.shape)

(590540, 434)
(590423, 434)


In [46]:
for i in range(11, 39) :
    col = 'id_'+str(i)
    if train[col].nunique() < 100 :
        a = list(set(train[col].unique()) - set(test[col].unique()))
        a = [x for x in a if str(x) != 'nan']
        tr2 = tr2.loc[~tr2[col].isin(a)]

In [47]:
print(tr2.shape)

(537419, 434)


In [48]:
for i in range(1, 340) :
    col = 'V'+str(i)
    if train[col].nunique() < 100 :
        a = list(set(train[col].unique()) - set(test[col].unique()))
        a = [x for x in a if str(x) != 'nan']
        tr2 = tr2.loc[~tr2[col].isin(a)]

In [49]:
print(tr2.shape)

(532734, 434)


In [50]:
for i in range(1, 10) :
    col = 'M'+str(i)
    if train[col].nunique() < 100 :
        a = list(set(train[col].unique()) - set(test[col].unique()))
        a = [x for x in a if str(x) != 'nan']
        tr2 = tr2.loc[~tr2[col].isin(a)]

In [51]:
print(tr2.shape)

(532734, 434)


In [52]:
for i in range(1, 16) :
    col = 'D'+str(i)
    if train[col].nunique() < 100 :
        a = list(set(train[col].unique()) - set(test[col].unique()))
        a = [x for x in a if str(x) != 'nan']
        tr2 = tr2.loc[~tr2[col].isin(a)]

In [53]:
print(tr2.shape)

(532734, 434)


In [54]:
for i in range(1, 15) :
    col = 'C'+str(i)
    if train[col].nunique() < 100 :
        a = list(set(train[col].unique()) - set(test[col].unique()))
        a = [x for x in a if str(x) != 'nan']
        tr2 = tr2.loc[~tr2[col].isin(a)]

In [55]:
print(tr2.shape)

(532732, 434)
